# OAE Atlas Dataset

This notebook downloads a curated subset of the Ocean Carbon Dioxide Removal (OAE) Atlas dataset to a local cache using the S3 access helpers in `cdr_atlas.py`. It also provides quick visual checks and documentation of what is being downloaded.

Dataset hosted on Source Cooperative: https://source.coop/cworthy/oae-efficiency-atlas

## Dataset summary

- The dataset contains results from thousands of CESM ocean alkalinity enhancement simulations.
- Files are organized by polygon (region), injection month, and simulation year/month.
- Each experiment includes perturbation variables and counterfactual variables (suffix `_ALT_CO2`).
- Polygon masks and efficiency maps are provided as NetCDF files.

Use this notebook to prefetch data in batches to avoid repeated downloads during analysis.

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import xarray as xr
import yaml

import cdr_atlas

/global/homes/m/mattlong/.conda/envs/atlas-calcs/lib/python3.13/site-packages/pop_tools/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


## Configuration

Choose a polygon ID, injection month, and a set of model years to download. The files follow:

```
experiments/{polygon_id}/{injection_month}/alk-forcing.{polygon_id}-{injection_year}-{injection_month}.pop.h.{year}-{month}.nc
```

`polygon_id` ranges from 000–690 and `injection_month` is typically 01/04/07/10 for the published experiments. Use `n_test` to limit the number of files pulled.

Model year 0347 corresponds to the calendar year 1999, thus we need a `model_year_offset` to align the model calendar with the true calendar year.

:::{note}
`n_test = None` to download the full temporal dataset range of the datasets at the specified 
`polygon_id`, `injection_year`, `injection_month`. 

If all these parameters are `None`, all available data will be downloaded.
:::

In [2]:
batch_size = 50

 # set any to None to download all available
polygon_id = 1
injection_year = 1999
injection_month = 1

# n_test limits the number of files downloaded
n_test = 3

dask_cluster_kwargs = {}


In [3]:
# Parameters
polygon_id = None
injection_year = 1999
injection_month = 1
n_test = None
dask_cluster_kwargs = {
    "account": "m4632",
    "queue_name": "premium",
    "n_nodes": 1,
    "n_tasks_per_node": 128,
    "wallclock": "06:00:00",
    "scheduler_file": None,
}


## Step 1: Identify files to download

We first enumerate all expected files for this polygon/time window. This avoids repeated S3 listings during computation.

In [4]:
atlas_data = cdr_atlas.DATASET_REGISTRY["oae-efficiency-map_atlas-v0"]
atlas_data.df

s3_path  \
injection_year polygon_id injection_month year month                                                      
1999           0          1               347  1      s3://us-west-2.opendata.source.coop/cworthy/oa...   
                                               2      s3://us-west-2.opendata.source.coop/cworthy/oa...   
                                               3      s3://us-west-2.opendata.source.coop/cworthy/oa...   
                                               4      s3://us-west-2.opendata.source.coop/cworthy/oa...   
                                               5      s3://us-west-2.opendata.source.coop/cworthy/oa...   
...                                                                                                 ...   
               689        10              362  5      s3://us-west-2.opendata.source.coop/cworthy/oa...   
                                               6      s3://us-west-2.opendata.source.coop/cworthy/oa...   
                                               7      s3://us-west-2.opendata.source.coop/cworthy/oa...   
                                               8      s3://us-west-2.opendata.source.coop/cworthy/oa...   
                                               9      s3://us-west-2.opendata.source.coop/cworthy/oa...   

                                                                                             cache_path  
injection_year polygon_id injection_month year month                                                     
1999           0          1               347  1      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
                                               2      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
                                               3      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
                                               4      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
                                               5      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
...                                                                                                 ...  
               689        10              362  5      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
                                               6      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
                                               7      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
                                               8      /pscratch/sd/m/mattlong/atlas_cache/experiment...  
                                               9      /pscratch/sd/m/mattlong/atlas_cache/experiment...  

[496800 rows x 2 columns]

In [5]:
manifest = atlas_data.query(polygon_id=polygon_id, injection_year=injection_year, injection_month=injection_month, n_test=n_test)
manifest

[{'s3_path': 's3://us-west-2.opendata.source.coop/cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-01.nc',
  'cache_path': '/pscratch/sd/m/mattlong/atlas_cache/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-01.nc'},
 {'s3_path': 's3://us-west-2.opendata.source.coop/cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-02.nc',
  'cache_path': '/pscratch/sd/m/mattlong/atlas_cache/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-02.nc'},
 {'s3_path': 's3://us-west-2.opendata.source.coop/cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-03.nc',
  'cache_path': '/pscratch/sd/m/mattlong/atlas_cache/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-03.nc'},
 {'s3_path': 's3://us-west-2.opendata.source.coop/cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-04.nc',
  'cache_path': '/pscratch/sd/m/mattlong/atlas_cache/experiments/000

## Step 2: Download missing files in batches

Downloads happen in batches to reduce overhead and better utilize S3 transfer throughput.

In [ ]:
downloaded = atlas_data.ensure_cache(polygon_id=polygon_id, injection_year=injection_year, injection_month=injection_month, n_test=n_test)

print(f"Downloaded {downloaded} file(s) to cache: {cdr_atlas.CACHE_DIR}")

## Step 3: Open cached data and inspect variables

This uses the cached NetCDF files and avoids any further network transfer.

In [ ]:
ds = atlas_data.open_dataset(polygon_id=polygon_id, injection_year=injection_year, injection_month=injection_month, n_test=n_test)
ds

## Plot 1: Polygon mask overview

The polygon masks dataset defines the spatial regions used in the experiments. This plot previews the polygon IDs on the global grid.

In [ ]:
ds_atlas_polygons = atlas_data.get_polygon_masks_dataset()
polygon_ids = ds_atlas_polygons.polygon_id
polygon_ids.plot()

## Plot 2: Example FG_CO2 slice

The dataset includes FG_CO2 and FG_ALT_CO2 variables (perturbation vs. counterfactual). This plots one slice for the selected polygon and injection date.

In [ ]:
injection_date = f"{injection_year:04d}-{injection_month:02d}"

fg_co2 = ds.FG_CO2.sel(
    polygon_id=polygon_id, injection_date=injection_date
).isel(elapsed_time=0)

fg_co2.plot()

## Example 3: Subset to a ROMS-Tools grid

Use a ROMS-Tools grid YAML to find which atlas polygons intersect your model domain. This is a lightweight spatial filter that you can use before downloading or integrating data.

In [ ]:
from parsers import load_roms_tools_object
domain_name = "cson_roms-marbl_v0.1_ccs-12km"

# Path to a ROMS-Tools grid YAML
grid_yaml = f"cson_forge/blueprints/{domain_name}/_grid.yml"
model_grid = load_roms_tools_object(grid_yaml)

model_grid.plot()


In [ ]:
# Create AtlasModelGridAnalyzer instance
analyzer = cdr_atlas.AtlasModelGridAnalyzer(model_grid, atlas_data)

# Get polygon IDs within model grid boundaries
print(f"Found {len(analyzer.polygon_ids_in_bounds)} unique polygon IDs within model grid boundaries")
print(f"Polygon IDs: {analyzer.polygon_ids_in_bounds[:100]}..." if len(analyzer.polygon_ids_in_bounds) > 100 else f"Polygon IDs: {analyzer.polygon_ids_in_bounds}")

analyzer.polygon_id_mask.plot(vmin=-1, vmax=analyzer.polygon_id_mask.max())

## Notes and references

- Data source and full README: https://source.coop/cworthy/oae-efficiency-atlas
- Cached files are stored under `cdr_atlas.CACHE_DIR` (defaults to `$SCRATCH/atlas_cache`).